In [1]:
%AddDeps com.datastax.spark spark-cassandra-connector_2.11 2.4.3
%AddDeps org.elasticsearch elasticsearch-spark-20_2.11 6.8.22
%AddDeps org.postgresql postgresql 42.3.3

Marking com.datastax.spark:spark-cassandra-connector_2.11:2.4.3 for download
Obtained 2 files


Waiting for a Spark session to start...

Marking org.elasticsearch:elasticsearch-spark-20_2.11:6.8.22 for download
Obtained 2 files


Waiting for a Spark session to start...

Marking org.postgresql:postgresql:42.3.3 for download
Obtained 2 files


In [2]:
import org.apache.spark.sql.{DataFrame, SparkSession, Column}

import org.postgresql.Driver
import java.sql.{Connection, DriverManager, Statement}

import org.apache.spark.sql.expressions.UserDefinedFunction
import org.apache.spark.sql.functions.{udf, col, sum, explode, lit}

import scala.collection.mutable.WrappedArray 
import java.net.{URL, URLDecoder}
import scala.util.Try

In [3]:
val spark: SparkSession = SparkSession.builder()
.config("spark.cassandra.connection.host", "10.0.0.31")
.config("spark.cassandra.connection.port", "9042")
.config("spark.jars", "/data/home/dmitriy.shirshov/postgresql-42.3.3.jar")
.appName("Shirshov scala k test")
.getOrCreate()

spark = org.apache.spark.sql.SparkSession@61533a53


org.apache.spark.sql.SparkSession@61533a53

In [4]:
val clients: DataFrame = spark.read
.format("org.apache.spark.sql.cassandra")
.options(Map("table" -> "clients", "keyspace" -> "labdata"))
.load()

clients = [uid: string, age: int ... 1 more field]


[uid: string, age: int ... 1 more field]

In [33]:
val logs: DataFrame = spark.read
.json("hdfs:///labs/laba03/weblogs.json")

logs = [uid: string, visits: array<struct<timestamp:bigint,url:string>>]


[uid: string, visits: array<struct<timestamp:bigint,url:string>>]

In [37]:
val visits: DataFrame = spark.read
.format("org.elasticsearch.spark.sql")
.options(Map("es.read.metadata" -> "true",
             "es.nodes.wan.only" -> "true",
             "es.port" -> "9200",
             "es.nodes" -> "10.0.0.31",
             "es.net.ssl" -> "false"))
.load("visits")

visits = [category: string, event_type: string ... 5 more fields]


[category: string, event_type: string ... 5 more fields]

In [26]:
val cats: DataFrame = spark.read
.format("jdbc")
.option("url", "jdbc:postgresql://10.0.0.31:5432/labdata")
.option("dbtable", "domain_cats")
.option("user", "dmitriy_shirshov")
.option("password", "qR4cxBAf")
.option("driver", "org.postgresql.Driver")
.load()

cats = [domain: string, category: string]


[domain: string, category: string]

In [99]:
val logsFiltered: DataFrame  = logs 
.withColumn("info", explode($"visits"))
.select("uid", "info")
.select("uid", "info.*")
.withColumn("target_domain", decodeUrlAndGetDomain(col("url")))
.drop("timestamp", "url")
.filter("target_domain != ''")

logsFiltered = [uid: string, target_domain: string]


lastException: Throwable = null


[uid: string, target_domain: string]

In [86]:
val clientsGrouped: DataFrame = clients
.withColumn("age_cat", getAgeGroup(col("age")))
.drop("age")

clientsGrouped = [uid: string, gender: string ... 1 more field]


[uid: string, gender: string ... 1 more field]

In [95]:
val logsCategory: DataFrame = logsFiltered
.join(cats, $"domain" === $"target_domain")
.select("uid", "category")
.withColumn("cat", webUdf($"category"))
.drop("category")

logsCategory = [uid: string, cat: string]


[uid: string, cat: string]

In [91]:
val visitsCategory: DataFrame = visits
.filter("uid is not null")
.withColumn("cat", catUdf($"category"))
.select("uid", "cat")

visitsCategory = [uid: string, cat: string]


[uid: string, cat: string]

In [92]:
val catsCategory: DataFrame = visitsCategory
.union(logsCategory)

catsCategory = [uid: string, cat: string]


[uid: string, cat: string]

In [100]:
val writeStage: DataFrame = clientsGrouped.alias("a")
.join(catsCategory.alias("b"),
      col("a.uid") === col("b.uid"),
      "left")
.withColumn("counter", lit(1))
.groupBy("a.uid", "a.gender", "a.age_cat")
.pivot("b.cat")
.agg(sum($"counter"))
.drop("null")

writeStage = [uid: string, gender: string ... 40 more fields]


[uid: string, gender: string ... 40 more fields]

In [101]:
writeStage.printSchema

root
 |-- uid: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age_cat: string (nullable = true)
 |-- shop_cameras: long (nullable = true)
 |-- shop_clothing: long (nullable = true)
 |-- shop_computers: long (nullable = true)
 |-- shop_cosmetics: long (nullable = true)
 |-- shop_entertainment_equipment: long (nullable = true)
 |-- shop_everyday_jewelry: long (nullable = true)
 |-- shop_house_repairs_paint_tools: long (nullable = true)
 |-- shop_household_appliances: long (nullable = true)
 |-- shop_household_furniture: long (nullable = true)
 |-- shop_kitchen_appliances: long (nullable = true)
 |-- shop_kitchen_utensils: long (nullable = true)
 |-- shop_luggage: long (nullable = true)
 |-- shop_mobile_phones: long (nullable = true)
 |-- shop_shoes: long (nullable = true)
 |-- shop_sports_equipment: long (nullable = true)
 |-- shop_toys: long (nullable = true)
 |-- web_arts_and_entertainment: long (nullable = true)
 |-- web_autos_and_vehicles: long (nullable = true)


In [102]:
writeStage.write
.format("jdbc")
.option("url", "jdbc:postgresql://10.0.0.31:5432/dmitriy_shirshov")
.option("dbtable", "clients")
.option("user", "dmitriy_shirshov")
.option("password", "qR4cxBAf")
.option("driver", "org.postgresql.Driver")
.option("truncate", value = true)
.mode("overwrite")
.save()

In [96]:
logsCategory
.filter(col("uid") === "d50192e5-c44e-4ae8-ae7a-7cfe67c8b777").collect()

Array([d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,web_news_and_media], [d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,web_news_and_media], [d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,web_news_and_media], [d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,web_news_and_media])

In [88]:
logsFiltered
.filter(col("uid") === "d50192e5-c44e-4ae8-ae7a-7cfe67c8b777").collect()

Array([d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,zebra-zoya.ru], [d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,news.yandex.ru], [d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,sotovik.ru], [d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,news.yandex.ru], [d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,sotovik.ru])

In [77]:
logs
.filter(col("uid") === "d50192e5-c44e-4ae8-ae7a-7cfe67c8b777").collect()

Array([d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,WrappedArray([1419688144068,http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun], [1426666298001,http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story], [1426666298000,http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html], [1426661722001,http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story], [1426661722000,http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html])])

In [176]:
spark.stop()

### Conf_functions 

In [97]:
object data_assembler {

  private def decodeUrlAndGetDomain: UserDefinedFunction = udf((url: String) => {

    Try {
      new URL(URLDecoder.decode(url, "UTF-8")).getHost.replace("www.", "")

    }
      .getOrElse("")
  })

  private def getAgeGroup: UserDefinedFunction = udf((age: Integer) => {

    age match {
      case a if a >= 18 && a <= 24 => "18-24"
      case a if a >= 25 && a <= 34 => "25-34"
      case a if a >= 35 && a <= 44 => "35-44"
      case a if a >= 45 && a <= 54 => "45-54"
      case a if a >= 55 => ">=55"
    }
  })

  private def webUdf: UserDefinedFunction = udf((web: String) => {

    "web_" + web

  })
  private def catUdf: UserDefinedFunction = udf((cat: String) => {

    "shop_" + cat.toLowerCase.split("-").mkString("_")

  })

  private def createTableWithGrant(shop_cols: Array[Any],
                           web_cols: Array[Any]): Unit = {

    val driverClass: Class[Driver] = classOf[org.postgresql.Driver]
    val driver: Any = Class.forName("org.postgresql.Driver").newInstance()
    val url = "jdbc:postgresql://10.0.0.31:5432/dmitriy_shirshov?user=dmitriy_shirshov&password=qR4cxBAf"
    val connection: Connection = DriverManager.getConnection(url)
    val statement: Statement = connection.createStatement()

    val query: String = ("create table if not exists clients (uid varchar(40), gender varchar(1), age_cat varchar(5), "
      + shop_cols.mkString(s" int, ") + " int, "
      + web_cols.mkString(s" int, ") + " int);")
    val bool_0: Boolean = statement.execute(query)

    val bool_1: Boolean = statement.execute("grant select on clients to labchecker2;")
    connection.close()
  }

  def data_assembler(): Unit = {

    val spark: SparkSession = SparkSession.builder()
      .config("spark.cassandra.connection.host", "10.0.0.31")
      .config("spark.cassandra.connection.port", "9042")
      .appName("Shirshov scala k test")
      .getOrCreate()

    val clients: DataFrame = spark.read
      .format("org.apache.spark.sql.cassandra")
      .options(Map("table" -> "clients", "keyspace" -> "labdata"))
      .load()

    val logs: DataFrame = spark.read
      .json("hdfs:///labs/laba03/weblogs.json")

    val visits: DataFrame = spark.read
      .format("org.elasticsearch.spark.sql")
      .options(Map("es.read.metadata" -> "true",
        "es.nodes.wan.only" -> "true",
        "es.port" -> "9200",
        "es.nodes" -> "10.0.0.31",
        "es.net.ssl" -> "false"))
      .load("visits")

    val shop_cols: Array[Any] = {
      visits
        .select("category")
        .distinct
        .withColumn("shop_item",
          catUdf(col("category")))
        .sort("shop_item")
        .rdd
        .map(v => v{1})
        .collect()
    }

    val cats: DataFrame = spark.read
      .format("jdbc")
      .option("url", "jdbc:postgresql://10.0.0.31:5432/labdata")
      .option("dbtable", "domain_cats")
      .option("user", "dmitriy_shirshov")
      .option("password", "qR4cxBAf")
      .option("driver", "org.postgresql.Driver")
      .load()

    val web_cols: Array[Any] = {
      cats
        .withColumn("web_category", webUdf(col("category")))
        .select("web_category")
        .distinct
        .sort("web_category")
        .rdd
        .map(v => v{0})
        .collect()
    }

    createTableWithGrant(shop_cols, web_cols)

    val logsFiltered: DataFrame = logs
      .withColumn("info", explode(col("visits")))
      .select("uid", "info")
      .select("uid", "info.*")
      .withColumn("target_domain", decodeUrlAndGetDomain(col("url")))
      .drop("timestamp", "url")
      .filter("target_domain != ''")

    val clientsGrouped: DataFrame = clients
      .withColumn("age_cat", getAgeGroup(col("age")))
      .drop("age")

    val logsCategory: DataFrame = logsFiltered
      .join(cats, col("domain") === col("target_domain"))
      .select("uid", "category")
      .withColumn("cat", webUdf(col("category")))
      .drop("category")

    val visitsCategory: DataFrame = visits
      .filter("uid is not null")
      .withColumn("cat", catUdf(col("category")))
      .select("uid", "cat")

    val catsCategory: DataFrame = visitsCategory
      .union(logsCategory)

    val writeStage: DataFrame = clientsGrouped.alias("a")
    .join(catsCategory.alias("b"),
          col("a.uid") === col("b.uid"),
          "left")
    .withColumn("counter", lit(1))
    .groupBy("a.uid", "a.gender", "a.age_cat")
    .pivot("b.cat")
    .agg(sum($"counter"))

    writeStage.write
      .format("jdbc")
      .option("url", "jdbc:postgresql://10.0.0.31:5432/dmitriy_shirshov")
      .option("dbtable", "clients")
      .option("user", "dmitriy_shirshov")
      .option("password", "qR4cxBAf")
      .option("driver", "org.postgresql.Driver")
      .option("truncate", value = true)
      .mode("overwrite")
      .save()

    spark.stop

  }
}

defined object data_assembler


In [ ]:
data_assembler.data_assembler()

### Initiate instance

In [139]:
val shop_cols: Array[Any] = {visits
                             .select("category")
                             .distinct
                             .withColumn("shop_item",
                                         catUdf(col("category")))
                             .sort("shop_item")
                             .rdd
                             .map(v => v{1})
                             .collect()}


val web_cols: Array[Any] = {cats
                            .withColumn("web_category", webUdf(col("category")))
                            .select("web_category")
                            .distinct
                            .sort("web_category")
                            .rdd
                            .map(v => v{0})
                            .collect()}

shop_cols = Array(shop_cameras, shop_clothing, shop_computers, shop_cosmetics, shop_entertainment_equipment, shop_everyday_jewelry, shop_house_repairs_paint_tools, shop_household_appliances, shop_household_furniture, shop_kitchen_appliances, shop_kitchen_utensils, shop_luggage, shop_mobile_phones, shop_shoes, shop_sports_equipment, shop_toys)
web_cols = Array(web_arts_and_entertainment, web_autos_and_vehicles, web_beauty_and_fitness, web_books_and_literature, web_business_and_industry, web_career_and_education, web_computer_and_electronics, web_finance, web_food_and_drink, web_gambling, web_games, web_health, web_home_and_garden, web_internet_and_telecom, web_law_and_government, web_news_and_media, web_pets_and_animals, web_recreation_and_hobbies, web_reference, ...


Array(web_arts_and_entertainment, web_autos_and_vehicles, web_beauty_and_fitness, web_books_and_literature, web_business_and_industry, web_career_and_education, web_computer_and_electronics, web_finance, web_food_and_drink, web_gambling, web_games, web_health, web_home_and_garden, web_internet_and_telecom, web_law_and_government, web_news_and_media, web_pets_and_animals, web_recreation_and_hobbies, web_reference, ...

In [166]:
"create table if not exists clients (uid varchar(40), gender varchar(1), age_cat integer, " + shop_cols.mkString(s" integer, ") + " integer, " + web_cols.mkString(s" integer, ")  + " integer);"

create table if not exists clients (uid varchar(40), gender varchar(1), age_cat integer, shop_cameras integer, shop_clothing integer, shop_computers integer, shop_cosmetics integer, shop_entertainment_equipment integer, shop_everyday_jewelry integer, shop_house_repairs_paint_tools integer, shop_household_appliances integer, shop_household_furniture integer, shop_kitchen_appliances integer, shop_kitchen_utensils integer, shop_luggage integer, shop_mobile_phones integer, shop_shoes integer, shop_sports_equipment integer, shop_toys integer, web_arts_and_entertainment integer, web_autos_and_vehicles integer, web_beauty_and_fitness integer, web_books_and_literature integer, web_business_and_industry integer, web_career_and_education integer, web_computer_and_electronics inte...

In [171]:
def createTableWithGrant(shop_cols: Array[Any],
                         web_cols: Array[Any]): Unit = {
    
    val driverClass: Class[Driver] = classOf[org.postgresql.Driver]
    val driver: Any = Class.forName("org.postgresql.Driver").newInstance()
    val url = "jdbc:postgresql://10.0.0.31:5432/dmitriy_shirshov?user=dmitriy_shirshov&password=qR4cxBAf"
    val connection: Connection = DriverManager.getConnection(url)
    val statement: Statement = connection.createStatement()
    
    val query: String = ("create table if not exists clients (uid varchar(40), gender varchar(1), age_cat varchar(5), " 
                         + shop_cols.mkString(s" int, ") + " int, " 
                         + web_cols.mkString(s" int, ")  + " int);")
    val bool_0: Boolean = statement.execute(query)
    
    val bool_1: Boolean = statement.execute("grant select on clients to labchecker2;")
    connection.close()
}

createTableWithGrant: (shop_cols: Array[Any], web_cols: Array[Any])Unit


In [172]:
createTableWithGrant(shop_cols, web_cols)